In [5]:
# %%

import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2 
import IPython
from glob import glob
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
import tqdm
#import seaborn as sns
import albumentations as A
import torch
import torch.nn as nn
import timm
import torcheval 
import wandb
import torchvision
from sklearn.model_selection import StratifiedKFold
from scipy import stats
import seaborn as sns
from albumentations.pytorch import ToTensorV2
plt.style.use('ggplot')
plt.rcParams.update(**{'figure.dpi':150})
import psutil
import imageio.v3 as imageio

# %%
RANDOM_NUMBER = 42
torch.manual_seed(RANDOM_NUMBER)

# %% [markdown]
# # select Device

# %%
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_DIRECTORY = os.path.join(os.getcwd(),"data")
torch.cuda.empty_cache()

# %%
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler , MinMaxScaler
import scipy as sp
#pid :279026 
# %%
class Config():

    BASE_DIR = os.path.join(os.getcwd() , 'data')
    train_df = pd.read_csv(BASE_DIR  +  '/train.csv')
    TRAIN_VAL_SPLIT_SIZE = 0.01
    TRAIN_BATCH_SIZE = 256
    VAL_BATCH_SIZE = 256
    TEST_BATCH_SIZE = 256
    LR_MAX = 1e-4 
    NUM_EPOCHS = 20
    IMG_OUT_FEATURES = 768  #512  # swin     #768<-vit   
    NORMALIZE_TARGET = "log_transform_mean_std"   #"log_transform" #
    RANDOM_NUMBER = 42
    NUM_FLODS  = 5
    NUM_CLASSES = 6
    TRAITS_NAME = ['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean' ]
    WANDB_INIT =  False
    FOLD = 0 # Which fold to set as validation data
    IMAGE_SIZE =128
    TARGET_IMAGE_SIZE =  224
    T_MAX =        9
    torch.manual_seed(RANDOM_NUMBER)
    INCLUDE_EXTRA_FEATURES = True
    EXTRA_FEATURES_NORMALIZATION = "standard_scalar"  #"min_max_normalization"  #
    WEIGHT_DECAY = 0.01
    TABULAR_NN_OUTPUT  =   128 # 256
    IMG_MODEL_NAME = "dinoV2" #"efficientnet_v2" # 
    IMG_FINED_TUNED_WEIGHT = None  #f'{BASE_DIR}/swinV2.pth'
    Lower_Quantile = 0.005
    Upper_Quantile = 0.985 #0.985
    # 
    EXTRA_COLOUMN = ['WORLDCLIM_BIO1_annual_mean_temperature',
       'WORLDCLIM_BIO12_annual_precipitation',
       'WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month',
       'WORLDCLIM_BIO15_precipitation_seasonality',
       'WORLDCLIM_BIO4_temperature_seasonality',
       'WORLDCLIM_BIO7_temperature_annual_range',
       'SOIL_bdod_0.5cm_mean_0.01_deg',
       'SOIL_bdod_100.200cm_mean_0.01_deg',
       'SOIL_bdod_15.30cm_mean_0.01_deg',
       'SOIL_bdod_30.60cm_mean_0.01_deg',
       'SOIL_bdod_5.15cm_mean_0.01_deg',
       'SOIL_bdod_60.100cm_mean_0.01_deg', 'SOIL_cec_0.5cm_mean_0.01_deg',
       'SOIL_cec_100.200cm_mean_0.01_deg',
       'SOIL_cec_15.30cm_mean_0.01_deg', 'SOIL_cec_30.60cm_mean_0.01_deg',
       'SOIL_cec_5.15cm_mean_0.01_deg', 'SOIL_cec_60.100cm_mean_0.01_deg',
       'SOIL_cfvo_0.5cm_mean_0.01_deg',
       'SOIL_cfvo_100.200cm_mean_0.01_deg',
       'SOIL_cfvo_15.30cm_mean_0.01_deg',
       'SOIL_cfvo_30.60cm_mean_0.01_deg',
       'SOIL_cfvo_5.15cm_mean_0.01_deg',
       'SOIL_cfvo_60.100cm_mean_0.01_deg',
       'SOIL_clay_0.5cm_mean_0.01_deg',
       'SOIL_clay_100.200cm_mean_0.01_deg',
       'SOIL_clay_15.30cm_mean_0.01_deg',
       'SOIL_clay_30.60cm_mean_0.01_deg',
       'SOIL_clay_5.15cm_mean_0.01_deg',
       'SOIL_clay_60.100cm_mean_0.01_deg',
       'SOIL_nitrogen_0.5cm_mean_0.01_deg',
       'SOIL_nitrogen_100.200cm_mean_0.01_deg',
       'SOIL_nitrogen_15.30cm_mean_0.01_deg',
       'SOIL_nitrogen_30.60cm_mean_0.01_deg',
       'SOIL_nitrogen_5.15cm_mean_0.01_deg',
       'SOIL_nitrogen_60.100cm_mean_0.01_deg',
       'SOIL_ocd_0.5cm_mean_0.01_deg', 'SOIL_ocd_100.200cm_mean_0.01_deg',
       'SOIL_ocd_15.30cm_mean_0.01_deg', 'SOIL_ocd_30.60cm_mean_0.01_deg',
       'SOIL_ocd_5.15cm_mean_0.01_deg', 'SOIL_ocd_60.100cm_mean_0.01_deg',
       'SOIL_ocs_0.30cm_mean_0.01_deg', 'SOIL_phh2o_0.5cm_mean_0.01_deg',
       'SOIL_phh2o_100.200cm_mean_0.01_deg',
       'SOIL_phh2o_15.30cm_mean_0.01_deg',
       'SOIL_phh2o_30.60cm_mean_0.01_deg',
       'SOIL_phh2o_5.15cm_mean_0.01_deg',
       'SOIL_phh2o_60.100cm_mean_0.01_deg',
       'SOIL_sand_0.5cm_mean_0.01_deg',
       'SOIL_sand_100.200cm_mean_0.01_deg',
       'SOIL_sand_15.30cm_mean_0.01_deg',
       'SOIL_sand_30.60cm_mean_0.01_deg',
       'SOIL_sand_5.15cm_mean_0.01_deg',
       'SOIL_sand_60.100cm_mean_0.01_deg',
       'SOIL_silt_0.5cm_mean_0.01_deg',
       'SOIL_silt_100.200cm_mean_0.01_deg',
       'SOIL_silt_15.30cm_mean_0.01_deg',
       'SOIL_silt_30.60cm_mean_0.01_deg',
       'SOIL_silt_5.15cm_mean_0.01_deg',
       'SOIL_silt_60.100cm_mean_0.01_deg', 'SOIL_soc_0.5cm_mean_0.01_deg',
       'SOIL_soc_100.200cm_mean_0.01_deg',
       'SOIL_soc_15.30cm_mean_0.01_deg', 'SOIL_soc_30.60cm_mean_0.01_deg',
       'SOIL_soc_5.15cm_mean_0.01_deg', 'SOIL_soc_60.100cm_mean_0.01_deg',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m1',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m1',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m1',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m1',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m1',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m10',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m10',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m10',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m10',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m10',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m11',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m11',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m11',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m11',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m11',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m12',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m12',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m12',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m12',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m12',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m2',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m2',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m2',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m2',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m2',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m3',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m3',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m3',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m3',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m3',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m4',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m4',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m4',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m4',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m4',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m5',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m5',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m5',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m5',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m5',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m6',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m6',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m6',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m6',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m6',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m7',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m7',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m7',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m7',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m7',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m8',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m8',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m8',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m8',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m8',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m9',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m9',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m9',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m9',
       'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m9',
       'VOD_C_2002_2018_multiyear_mean_m01',
       'VOD_C_2002_2018_multiyear_mean_m02',
       'VOD_C_2002_2018_multiyear_mean_m03',
       'VOD_C_2002_2018_multiyear_mean_m04',
       'VOD_C_2002_2018_multiyear_mean_m05',
       'VOD_C_2002_2018_multiyear_mean_m06',
       'VOD_C_2002_2018_multiyear_mean_m07',
       'VOD_C_2002_2018_multiyear_mean_m08',
       'VOD_C_2002_2018_multiyear_mean_m09',
       'VOD_C_2002_2018_multiyear_mean_m10',
       'VOD_C_2002_2018_multiyear_mean_m11',
       'VOD_C_2002_2018_multiyear_mean_m12',
       'VOD_Ku_1987_2017_multiyear_mean_m01',
       'VOD_Ku_1987_2017_multiyear_mean_m02',
       'VOD_Ku_1987_2017_multiyear_mean_m03',
       'VOD_Ku_1987_2017_multiyear_mean_m04',
       'VOD_Ku_1987_2017_multiyear_mean_m05',
       'VOD_Ku_1987_2017_multiyear_mean_m06',
       'VOD_Ku_1987_2017_multiyear_mean_m07',
       'VOD_Ku_1987_2017_multiyear_mean_m08',
       'VOD_Ku_1987_2017_multiyear_mean_m09',
       'VOD_Ku_1987_2017_multiyear_mean_m10',
       'VOD_Ku_1987_2017_multiyear_mean_m11',
       'VOD_Ku_1987_2017_multiyear_mean_m12',
       'VOD_X_1997_2018_multiyear_mean_m01',
       'VOD_X_1997_2018_multiyear_mean_m02',
       'VOD_X_1997_2018_multiyear_mean_m03',
       'VOD_X_1997_2018_multiyear_mean_m04',
       'VOD_X_1997_2018_multiyear_mean_m05',
       'VOD_X_1997_2018_multiyear_mean_m06',
       'VOD_X_1997_2018_multiyear_mean_m07',
       'VOD_X_1997_2018_multiyear_mean_m08',
       'VOD_X_1997_2018_multiyear_mean_m09',
       'VOD_X_1997_2018_multiyear_mean_m10',
       'VOD_X_1997_2018_multiyear_mean_m11',
       'VOD_X_1997_2018_multiyear_mean_m12'
       ]

    N_TARGETS  =len(TRAITS_NAME)  


CONFIG = Config()



# %%
if CONFIG.WANDB_INIT:
    wandb.login()
    wandb.init(project="cs680v3",group="dino_V2",name="submission_dinoV2_no_fine_tune",
            config = {
        "LR_max": CONFIG.LR_MAX,
        "WEIGHT_DECAY":CONFIG.WEIGHT_DECAY,
        "train_batch" : CONFIG.TRAIN_BATCH_SIZE
        })


In [6]:

# %% [markdown]
# # Preprocessing the Tabular Data And Image Transformation

# %%
# define_transformation for the tabular data
log_tf_col = CONFIG.TRAITS_NAME
scale_feature_col =  CONFIG.EXTRA_COLOUMN + CONFIG.TRAITS_NAME
log_transform = ColumnTransformer(transformers= [
    ('log' , FunctionTransformer( np.log10  , inverse_func=sp.special.exp10, validate=False, check_inverse = True ,feature_names_out='one-to-one') , log_tf_col)
        ] , verbose_feature_names_out=False ,remainder= 'passthrough'
          )
log_transform.set_output(transform='pandas')

std_scale =  ColumnTransformer(transformers=[('scale',StandardScaler() ,scale_feature_col )  ],
                               verbose_feature_names_out=False,
                               remainder='passthrough'
                               )
std_scale.set_output(transform='pandas')
scaling_pipeline = Pipeline(steps=[   
                        ("log" , log_transform),
                        ("std_scale" , std_scale )])



# %%
# preparing the tabular data that has been given 
BASE_DIR = CONFIG.BASE_DIR
Train_DF = pd.read_csv(BASE_DIR  +  '/train.csv')
Test_DF =  pd.read_csv(BASE_DIR  +  '/test.csv')
Test_DF[log_tf_col] =1

train_df , val_df = train_test_split(Train_DF,test_size=CONFIG.TRAIN_VAL_SPLIT_SIZE,shuffle=True)
for column in CONFIG.TRAITS_NAME:
    lower_quantile = train_df[column].quantile(CONFIG.Lower_Quantile)
    upper_quantile = train_df[column].quantile(CONFIG.Upper_Quantile)
    train_df = train_df[(train_df[column] >= lower_quantile) & (train_df[column] <= upper_quantile)]

# train_df = train_df[(np.abs(stats.zscore(np.log10(train_df[CONFIG.TRAITS_NAME]))<2).all(axis=1)  )]

print('JPEG Files Processing:')
train_df['file_path'] = train_df['id'].apply(lambda s: f'{BASE_DIR}/train_images/{s}.jpeg')
train_df['jpeg_bytes'] = train_df['file_path'].apply(lambda fp: open(fp, 'rb').read())

val_df['file_path'] = val_df['id'].apply(lambda s: f'{BASE_DIR}/train_images/{s}.jpeg')
val_df['jpeg_bytes'] = val_df['file_path'].apply(lambda fp: open(fp, 'rb').read())

Test_DF['file_path'] = Test_DF['id'].apply(lambda s: f'{BASE_DIR}/test_images/{s}.jpeg')
Test_DF['jpeg_bytes'] = Test_DF['file_path'].apply(lambda fp: open(fp, 'rb').read())
print('JPEG Files Processing End')    


# train_tabular = train_df.drop(columns = ['id'] + CONFIG.TRAITS_NAME)
# val_tabular =    val_df.drop(columns = ['id'] + CONFIG.TRAITS_NAME)
# test_tabular = test.drop(columns = ['id'])

# scaling of the training data !

training_scaling_pipeline = Pipeline(steps=[   
                        ("log" , log_transform),
                        ("std_scale" , std_scale )])

train_tabular_scaled =  training_scaling_pipeline.fit_transform(train_df)
validation_tabular_scaled = training_scaling_pipeline.transform(val_df)

test_tabular_scaled = training_scaling_pipeline.transform(Test_DF)
Test_DF[log_tf_col]
"inverse"
# tt =   scaling_pipeline['std_scale']['scale'].inverse_transform(train_tabular_scaled[CONFIG.EXTRA_COLOUMN + CONFIG.TRAITS_NAME])
# tt = pd.DataFrame(tt, columns=CONFIG.EXTRA_COLOUMN + CONFIG.TRAITS_NAME)
# tt2 = scaling_pipeline['log']['log'].inverse_transform(tt[CONFIG.TRAITS_NAME])
# tt2  = pd.concat([tt2 ,tt[CONFIG.EXTRA_COLOUMN]],axis=1)                 
                  


# some hyper parameter for lr scheduler
CONFIG.N_TRAIN_SAMPLES = len(train_df)
CONFIG.N_STEPS_PER_EPOCH = (CONFIG.N_TRAIN_SAMPLES // CONFIG.TRAIN_BATCH_SIZE)
CONFIG.N_STEPS = CONFIG.N_STEPS_PER_EPOCH * CONFIG.NUM_EPOCHS + 1   

# %%
# Image PreProcessing

MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
TRAIN_TRANSFORMS = A.Compose([
    A.Resize(CONFIG.TARGET_IMAGE_SIZE,CONFIG.TARGET_IMAGE_SIZE),
    A.RandomResizedCrop(size=(CONFIG.TARGET_IMAGE_SIZE,CONFIG.TARGET_IMAGE_SIZE), interpolation=cv2.INTER_CUBIC,p=0.4),  # Simulate different crops
    A.HorizontalFlip(p=0.2),  
    A.ToFloat(),
    A.Normalize(mean=MEAN, std=STD, max_pixel_value=1),
    ToTensorV2(),
])

TEST_TRANSFORMS = A.Compose([
    A.Resize(CONFIG.TARGET_IMAGE_SIZE,CONFIG.TARGET_IMAGE_SIZE),
    #A.CenterCrop(CONFIG.TARGET_IMAGE_SIZE,CONFIG.TARGET_IMAGE_SIZE),
    A.ToFloat(),
    A.Normalize(mean=MEAN, std=STD, max_pixel_value=1),
    ToTensorV2(),
])



# %% [markdown]
# # Creating the Dataloader for Train , Validation and Testing 

# %%
class create_dataset(Dataset):
    def __init__(self, X_jpeg_bytes, X_tabular, y, transforms=None):
        self.X_jpeg_bytes = X_jpeg_bytes
        self.X_tabular = X_tabular
        self.y = y
        self.transforms = transforms

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        X_sample = self.transforms(
            image=imageio.imread(self.X_jpeg_bytes[index]),
        )['image']
        X_tabular_sample = self.X_tabular[index]
        y_sample = self.y[index]

        return X_sample, X_tabular_sample, y_sample




train_dataset = create_dataset(
    X_jpeg_bytes = train_tabular_scaled['jpeg_bytes'].values,
    X_tabular = train_tabular_scaled[CONFIG.EXTRA_COLOUMN].values.astype(np.float32),
    y = train_tabular_scaled[CONFIG.TRAITS_NAME].values.astype(np.float32),
    transforms= TRAIN_TRANSFORMS
)

validation_dataset = create_dataset(
    X_jpeg_bytes = validation_tabular_scaled['jpeg_bytes'].values,
    X_tabular = validation_tabular_scaled[CONFIG.EXTRA_COLOUMN].values.astype(np.float32),
    y = validation_tabular_scaled[CONFIG.TRAITS_NAME].values.astype(np.float32),
    transforms= TEST_TRANSFORMS
)

test_dataset = create_dataset(
    X_jpeg_bytes = test_tabular_scaled['jpeg_bytes'].values,
    X_tabular = test_tabular_scaled[CONFIG.EXTRA_COLOUMN].values.astype(np.float32),
    y = test_tabular_scaled["id"].values,
    transforms= TEST_TRANSFORMS
    )

train_dataloader = DataLoader(
    train_dataset,
    batch_size=CONFIG.TRAIN_BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    num_workers=0#psutil.cpu_count(),
)

validation_dataloader = DataLoader(
    validation_dataset,
    batch_size=CONFIG.VAL_BATCH_SIZE,
    shuffle=False,
    drop_last=False,
    num_workers= 0 #psutil.cpu_count(),
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=CONFIG.TEST_BATCH_SIZE,
    shuffle=False,
    drop_last=False,
    num_workers= 0 #psutil.cpu_count(),
)

JPEG Files Processing:
JPEG Files Processing End


In [7]:
X_tabular = train_tabular_scaled[CONFIG.EXTRA_COLOUMN].values.astype(np.float32)
y_tabular = train_tabular_scaled[CONFIG.TRAITS_NAME].values.astype(np.float32)
X_val_tabular = validation_tabular_scaled[CONFIG.EXTRA_COLOUMN].values.astype(np.float32)
y_val_tabular = validation_tabular_scaled[CONFIG.TRAITS_NAME].values.astype(np.float32)

In [8]:
import pandas as pd
import numpy  as np
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold

In [13]:
kf = KFold(n_splits=3, random_state=42, shuffle=True)
predictions_array =[]
CV_score_array    =[]
for train_index, test_index in kf.split(X_tabular):
    X_train, X_valid = X_tabular[train_index], X_tabular[test_index]
    y_train, y_valid = y_tabular[train_index], y_tabular[test_index]
    regressor = TabNetRegressor(verbose=1,seed=42 , device_name='cuda')
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              patience=300, max_epochs=2000,
              eval_metric=['rmse','mae'])
    CV_score_array.append(regressor.best_cost)
    predictions_array.append(np.expm1(regressor.predict(X_val_tabular)))

predictions = np.mean(predictions_array,axis=0)

/home/prajwal/anaconda3/envs/gpu2/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.30965 | val_0_rmse: 0.999530017375946| val_0_mae: 0.7223299741744995|  0:00:00s
epoch 1  | loss: 0.98249 | val_0_rmse: 0.9751899838447571| val_0_mae: 0.7144899964332581|  0:00:00s
epoch 2  | loss: 0.94495 | val_0_rmse: 0.9577299952507019| val_0_mae: 0.6805099844932556|  0:00:01s
epoch 3  | loss: 0.9178  | val_0_rmse: 0.9482899904251099| val_0_mae: 0.6784700155258179|  0:00:01s
epoch 4  | loss: 0.90612 | val_0_rmse: 0.9437900185585022| val_0_mae: 0.679889976978302|  0:00:02s
epoch 5  | loss: 0.89654 | val_0_rmse: 0.940310001373291| val_0_mae: 0.6613500118255615|  0:00:02s
epoch 6  | loss: 0.89005 | val_0_rmse: 0.9385499954223633| val_0_mae: 0.6716499924659729|  0:00:03s
epoch 7  | loss: 0.88209 | val_0_rmse: 0.9341800212860107| val_0_mae: 0.6620500087738037|  0:00:03s
epoch 8  | loss: 0.87637 | val_0_rmse: 0.93163001537323| val_0_mae: 0.6663399934768677|  0:00:04s
epoch 9  | loss: 0.87115 | val_0_rmse: 0.9306700229644775| val_0_mae: 0.660319983959198|  0:00:04s
epoch 

/home/prajwal/anaconda3/envs/gpu2/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/home/prajwal/anaconda3/envs/gpu2/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.2902  | val_0_rmse: 0.9995200037956238| val_0_mae: 0.7252699732780457|  0:00:00s
epoch 1  | loss: 0.97956 | val_0_rmse: 0.979200005531311| val_0_mae: 0.7091900110244751|  0:00:00s
epoch 2  | loss: 0.93138 | val_0_rmse: 0.9545199871063232| val_0_mae: 0.683210015296936|  0:00:01s
epoch 3  | loss: 0.90329 | val_0_rmse: 0.9470099806785583| val_0_mae: 0.680429995059967|  0:00:01s
epoch 4  | loss: 0.89381 | val_0_rmse: 0.9422500133514404| val_0_mae: 0.6664199829101562|  0:00:02s
epoch 5  | loss: 0.88679 | val_0_rmse: 0.9409599900245667| val_0_mae: 0.6772699952125549|  0:00:02s
epoch 6  | loss: 0.88244 | val_0_rmse: 0.9377599954605103| val_0_mae: 0.6603500247001648|  0:00:03s
epoch 7  | loss: 0.87845 | val_0_rmse: 0.935920000076294| val_0_mae: 0.6642400026321411|  0:00:03s
epoch 8  | loss: 0.875   | val_0_rmse: 0.9332699775695801| val_0_mae: 0.6627699732780457|  0:00:04s
epoch 9  | loss: 0.86892 | val_0_rmse: 0.9334700107574463| val_0_mae: 0.6565499901771545|  0:00:04s
epoc

/home/prajwal/anaconda3/envs/gpu2/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/home/prajwal/anaconda3/envs/gpu2/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.29311 | val_0_rmse: 1.000290036201477| val_0_mae: 0.7261800169944763|  0:00:00s
epoch 1  | loss: 0.96385 | val_0_rmse: 0.9599699974060059| val_0_mae: 0.6876299977302551|  0:00:00s
epoch 2  | loss: 0.90416 | val_0_rmse: 0.943120002746582| val_0_mae: 0.6698099970817566|  0:00:01s
epoch 3  | loss: 0.88068 | val_0_rmse: 0.9326500296592712| val_0_mae: 0.6608399748802185|  0:00:01s
epoch 4  | loss: 0.86976 | val_0_rmse: 0.9305400252342224| val_0_mae: 0.6556100249290466|  0:00:02s
epoch 5  | loss: 0.86574 | val_0_rmse: 0.9286500215530396| val_0_mae: 0.6529200077056885|  0:00:02s
epoch 6  | loss: 0.86381 | val_0_rmse: 0.9270700216293335| val_0_mae: 0.6621999740600586|  0:00:03s
epoch 7  | loss: 0.86102 | val_0_rmse: 0.9270200133323669| val_0_mae: 0.6537100076675415|  0:00:03s
epoch 8  | loss: 0.85636 | val_0_rmse: 0.924530029296875| val_0_mae: 0.6473600268363953|  0:00:03s
epoch 9  | loss: 0.857   | val_0_rmse: 0.9254900217056274| val_0_mae: 0.654699981212616|  0:00:04s
epoc

/home/prajwal/anaconda3/envs/gpu2/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [18]:
tt = regressor.predict(X_val_tabular)

In [20]:
tt

array([[-0.7123754 ,  0.4135599 , -0.5576975 , -0.36195028, -0.10042072,
        -0.41677377],
       [-0.42096737,  0.09176306, -0.3975855 , -0.24106547,  0.11140418,
        -0.42222494],
       [ 0.11245253,  0.13277547, -0.318211  , -0.27956572, -0.31234303,
        -0.13768162],
       ...,
       [ 0.5730102 , -0.8745638 ,  0.10610379, -0.24135126,  0.52413297,
        -0.43531868],
       [-0.6031854 ,  0.32449147, -0.50937974, -0.32487035, -0.065247  ,
        -0.4058866 ],
       [-0.02134127,  0.28702798, -0.00350395, -0.20220855, -0.3598288 ,
         0.07855164]], dtype=float32)

In [21]:
from sklearn.metrics import accuracy_score ,r2_score
print('Accuracy', r2_score(y_pred=tt, y_true=y_val_tabular))

Accuracy 0.14717154204845428


In [1]:
    EXTRA_COLOUMN = [
    'WORLDCLIM_BIO1_annual_mean_temperature',
'WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month',
'WORLDCLIM_BIO15_precipitation_seasonality',
'WORLDCLIM_BIO4_temperature_seasonality',
'SOIL_ocd_100.200cm_mean_0.01_deg',
'WORLDCLIM_BIO12_annual_precipitation',
'WORLDCLIM_BIO7_temperature_annual_range',
'SOIL_cec_0.5cm_mean_0.01_deg',
'SOIL_nitrogen_0.5cm_mean_0.01_deg',
'SOIL_ocd_60.100cm_mean_0.01_deg',
'SOIL_ocd_30.60cm_mean_0.01_deg',
'SOIL_clay_100.200cm_mean_0.01_deg',
'SOIL_soc_100.200cm_mean_0.01_deg',
'SOIL_nitrogen_100.200cm_mean_0.01_deg',
'SOIL_cec_100.200cm_mean_0.01_deg',
'SOIL_clay_15.30cm_mean_0.01_deg',
'SOIL_soc_60.100cm_mean_0.01_deg',
'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m10',
'SOIL_nitrogen_30.60cm_mean_0.01_deg',
'SOIL_clay_5.15cm_mean_0.01_deg',
'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m4',
'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m11',
'SOIL_ocd_0.5cm_mean_0.01_deg',
'SOIL_ocd_5.15cm_mean_0.01_deg',
'SOIL_cfvo_100.200cm_mean_0.01_deg',
'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m5',
'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m6',
'SOIL_cfvo_30.60cm_mean_0.01_deg',
'SOIL_cfvo_0.5cm_mean_0.01_deg',
'SOIL_soc_15.30cm_mean_0.01_deg',
'SOIL_nitrogen_15.30cm_mean_0.01_deg',
'SOIL_nitrogen_5.15cm_mean_0.01_deg',
'SOIL_clay_0.5cm_mean_0.01_deg',
'SOIL_nitrogen_60.100cm_mean_0.01_deg',
'SOIL_soc_0.5cm_mean_0.01_deg',
'SOIL_ocd_15.30cm_mean_0.01_deg',
'SOIL_ocs_0.30cm_mean_0.01_deg',
'SOIL_clay_30.60cm_mean_0.01_deg',
'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m3',
'SOIL_cfvo_15.30cm_mean_0.01_deg',
'SOIL_silt_100.200cm_mean_0.01_deg',
'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m8',
'SOIL_soc_30.60cm_mean_0.01_deg',
'SOIL_cfvo_5.15cm_mean_0.01_deg',
'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m12',
'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m10',
'SOIL_silt_0.5cm_mean_0.01_deg',
'MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m7',
'SOIL_sand_100.200cm_mean_0.01_deg',
'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m3',
'MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m12'
]

In [2]:
len(EXTRA_COLOUMN)

51